In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pylab as pylab
from datetime import datetime
import datetime as dtt
import os
import sys
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller

import astropy.stats as ast_stats

import pyflux as pf

import traces

import warnings
warnings.filterwarnings('ignore')
import tqdm
import copy

from sklearn.metrics import mean_absolute_error, mean_squared_error

import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
from scipy.optimize import minimize
from scipy import integrate

from sklearn.model_selection import TimeSeriesSplit

from scipy import integrate

from numpy import array, arange, abs as np_abs
from numpy.fft import rfft, rfftfreq

import stldecompose

from sklearn.manifold import TSNE,MDS
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.mixture import BayesianGaussianMixture

import pywt

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

from sklearn.preprocessing import MinMaxScaler

import heapq

import pprint

import math
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from  sklearn.model_selection import train_test_split

pd.options.display.max_columns=None
pd.options.display.max_rows=None

/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
Using TensorFlow backend.


# Новые данные (27.03.2018)

In [2]:
dir_path = './2018/'

In [3]:
def dive_into_folder(root):
    global attrs
    for folder in os.listdir(root):
        if os.path.isdir(os.path.join(root, folder)):
            attrs[folder] = {}
            get_data(os.path.join(root, folder), attrs, folder)

In [4]:
def get_ip_and_attr(string):
    vals = string.split('.')
    ip = []
    for val in vals:
        try:
            int(val)
            ip.append(val)
        except:
            return '.'.join(ip), val

In [5]:
def get_data(path, attrs, folder):
    for f in os.listdir(path):
        if os.path.isdir(os.path.join(path, f)):
            get_data(os.path.join(path, f), attrs, folder)
        else:
            if 'DS' in os.path.join(path, f):
                continue
            ip,val = get_ip_and_attr(f)
            if ip not in attrs[folder].keys():
                attrs[folder][ip] = {}
            ff = pd.read_csv(
                os.path.join(path, f),
                sep=',',
                encoding='utf-8',
                names=['timestamp', val]
            )
            ff['timestamp'] = ff['timestamp'].apply(lambda x: datetime.fromtimestamp(x/1000000))
            ff[val] = ff[val].astype(float)
            if ff.shape[0] > 10:
                s = pd.Series(data=ff[val].values, index=ff['timestamp'].values)
                index = s.resample('5T').mean().index.values
                s = s.resample('5T').mean().interpolate('linear')
                s = pd.Series(data=s.values, index=index)
                
                coeff , freq = pywt.cwt(s, np.arange(1,30), 'gaus1')
                attrs[folder][ip][val] = {
                    'df': ff,
                    'ts': s,
                    'coeff': coeff,
                    'freq': freq
                }
            else:
                attrs[folder][ip][val] = {
                    'df': ff,
                    'ts': -1,
                    'coeff': -1,
                    'freq': -1
                }

    return attrs

In [6]:
attrs = {}

In [7]:
%%time
dive_into_folder(dir_path)

CPU times: user 15min 35s, sys: 1min 23s, total: 16min 58s
Wall time: 32min 57s


In [8]:
attrs['dslam']['10.10.104.106'].keys()

dict_keys(['inUtilization', 'outUtilization'])

In [9]:
attrs['dslam']['10.10.104.106']['inUtilization']['df'].head()

,timestamp,inUtilization
0,2017-09-06 14:40:28.831607,1.92
1,2017-09-06 14:44:04.605664,2.74
2,2017-09-06 14:49:05.058544,1.87
3,2017-09-06 14:54:05.503277,1.78
4,2017-09-06 14:59:05.952444,1.21


In [10]:
def get_week_interval(df):
    time = 'timestamp'
    value = df.columns[-1]

    week_mapper = {
        range(1,8): 0,
        range(8,15): 1,
        range(15, 22): 2,
        range(22, 29): 3,
        range(29, 32): 4
    }
    week_2_arr = {}
    for k, v in week_mapper.items():
        vv = df[df[time].apply(lambda x: x.day).isin(k)][value].values
        week_2_arr[v] = vv if vv.shape[0] else np.asarray([0]) 

    return week_2_arr

In [11]:
def get_days_interval(df):
    time = 'timestamp'
    value = df.columns[-1]
    day_2_arr = {}
    dd = df.copy(deep=True)
    week_days = dd[time].apply(lambda x: x.weekday())
    for j in range(7):
        mask = week_days == j
        day_2_arr[j] = np.asarray(dd.loc[mask, value].values) if mask.shape[0] != 0 else np.asarray([0])
    return day_2_arr

In [12]:
def get_hour_intervals(df):
    time = 'timestamp'
    value = df.columns[-1]
    hour_2_arr = {}
    dd = df.copy(deep=True)
    hours = dd[time].apply(lambda x: x.hour)
    for j in range(24):
        mask = hours == j
        hour_2_arr[j] = np.asarray(dd.loc[mask, value].values) if mask.shape[0] != 0 else np.asarray([0])
    return hour_2_arr

In [13]:
def get_median_stat_per_week(vals, stat_func, perc_num=0, integral=False):
    res = []
    for interval in range(5):
        value = vals[interval]
        if perc_num:
            res.append(stat_func(value, perc_num))
        elif not integral:
            res.append(stat_func(value))
        else:
            res.append(stat_func(y=value)/value.shape[0])

    return res #np.median(np.asarray(res))

In [14]:
def get_median_stat_per_days(vals, stat_func, perc_num=0, integral=False):
    vv = []
    for k in range(7):
        if k in vals.keys():
            if vals[k].shape[0] > 1:
                if perc_num:
                    vv.append(stat_func(vals[k], perc_num))
                else:
                    if not integral:
                        vv.append(stat_func(vals[k]))
                    else:
                        vv.append(stat_func(y=vals[k])/vals[k].shape[0])
            else:
                vv.append(0)
        else:
            vv.append(0)
        
    return vv #np.median(np.asarray(vv))

In [15]:
def get_median_stat_per_hours(vals, stat_func, perc_num=0, integral=False):
    vv = []
    for k in range(24):
        if k in vals.keys():
            if vals[k].shape[0] > 1:
                if perc_num:
                    vv.append(stat_func(vals[k], perc_num))
                else:
                    if not integral:
                        vv.append(stat_func(vals[k]))
                    else:
                        vv.append(stat_func(y=vals[k])/vals[k].shape[0])
            else:
                vv.append(0)
        else:
            vv.append(0)
    return vv #np.median(np.asarray(vv))

In [16]:
def get_col_name(cols, k):
    for col in cols:
        if k in col.lower():
            return col
        
def get_preliminare_stat(attrs):
    days = 7
    params = 8
    hourss = 24
    weeks = 5
    #'model', 'ip', 'cpu_length', 'cpu_min', 'cpu_max', 'cpu_mean', 'cpu_median', 'cpu_std', cpu_unique
    label = None
    a = None
    mapper =  {'cpu':0, 'mem':1,'outut':2, 'inut':3}
    for model, model_dict in attrs.items():
        print(model)
        for ip, ip_dict in model_dict.items():
            vv = [
                np.asarray([-1 for _ in range(weeks*params + days*params + hourss*params+2)])
                for i in range(len(list(mapper.keys())))
            ]
            for val, val_dict in ip_dict.items():
                df = val_dict['df']
                for k, v in mapper.items():
                    if k in val.lower():
                        col_name = get_col_name(df.columns, k)
                        if df[col_name].shape[0] > 0:
                            data_day = get_days_interval(df)
                            data_week = get_week_interval(df)
                            data_hour = get_hour_intervals(df)
                            
                            row_week = np.asarray(
                                get_median_stat_per_week(data_week, np.min) +
                                get_median_stat_per_week(data_week, np.max) +
                                get_median_stat_per_week(data_week, np.mean) +
                                get_median_stat_per_week(data_week, np.median) +
                                get_median_stat_per_week(data_week, np.std) +
                                get_median_stat_per_week(data_week, np.percentile, 25) +
                                get_median_stat_per_week(data_week, np.percentile, 75) +
                                get_median_stat_per_week(data_week, integrate.simps, integral=True)
                            )
                            row_days = np.asarray(
                                get_median_stat_per_days(data_day, np.min) +
                                get_median_stat_per_days(data_day, np.max) +
                                get_median_stat_per_days(data_day, np.mean) +
                                get_median_stat_per_days(data_day, np.median) +
                                get_median_stat_per_days(data_day, np.std) +
                                get_median_stat_per_days(data_day, np.percentile, 25) +
                                get_median_stat_per_days(data_day, np.percentile, 75) +
                                get_median_stat_per_days(data_day, integrate.simps, integral=True)
                            )
                            row_hours = np.asarray(
                                get_median_stat_per_hours(data_hour, np.min) +
                                get_median_stat_per_hours(data_hour, np.max) +
                                get_median_stat_per_hours(data_hour, np.mean) +
                                get_median_stat_per_hours(data_hour, np.median) +
                                get_median_stat_per_hours(data_hour, np.std) +
                                get_median_stat_per_hours(data_hour, np.percentile, 25) +
                                get_median_stat_per_hours(data_hour, np.percentile, 75) +
                                get_median_stat_per_hours(data_hour, integrate.simps, integral=True)
                            )
                            datess = np.asarray([df['timestamp'].min(),df['timestamp'].max()])
                            vv[v] = np.hstack([row_week, row_days, row_hours, datess])
                            break

#                         for lag in range(1,10):
#                             clust_data['{}_lag{}'.format(preffix,lag)] = cpu.groupby('ip').apply(lambda v: v[preffix].autocorr(lag=lag))

            row = np.append(np.asarray([model,ip]), np.asarray(vv).flatten())
            a = row if a is None else np.vstack((a, row))
            
    cols = ['model', 'ip']
    for col in ['cpu', 'mem','inUtil', 'outUtil']:
        i = ['{}_week_min_{}'.format(col,i) for i in range(weeks)] + \
            ['{}_week_max_{}'.format(col,i) for i in range(weeks)] + \
            ['{}_week_mean_{}'.format(col,i) for i in range(weeks)] + \
            ['{}_week_median_{}'.format(col,i) for i in range(weeks)] + \
            ['{}_week_std_{}'.format(col,i) for i in range(weeks)] + \
            ['{}_week_q25_{}'.format(col,i) for i in range(weeks)] + \
            ['{}_week_q75_{}'.format(col,i) for i in range(weeks)] + \
            ['{}_week_integ_{}'.format(col,i) for i in range(weeks)] + \
            ['{}_days_min_{}'.format(col,i) for i in range(days)] + \
            ['{}_days_max_{}'.format(col,i) for i in range(days)] + \
            ['{}_days_mean_{}'.format(col,i) for i in range(days)] + \
            ['{}_days_median_{}'.format(col,i) for i in range(days)] + \
            ['{}_days_std_{}'.format(col,i) for i in range(days)] + \
            ['{}_days_q25_{}'.format(col,i) for i in range(days)] + \
            ['{}_days_q75_{}'.format(col,i) for i in range(days)] + \
            ['{}_days_integ_{}'.format(col,i) for i in range(days)] + \
            ['{}_hours_min_{}'.format(col,i) for i in range(hourss)] + \
            ['{}_hours_max_{}'.format(col,i) for i in range(hourss)] + \
            ['{}_hours_mean_{}'.format(col,i) for i in range(hourss)] + \
            ['{}_hours_median_{}'.format(col,i) for i in range(hourss)] + \
            ['{}_hours_std_{}'.format(col,i) for i in range(hourss)] + \
            ['{}_hours_q25_{}'.format(col,i) for i in range(hourss)] + \
            ['{}_hours_q75_{}'.format(col,i) for i in range(hourss)] + \
            ['{}_hours_integ_{}'.format(col,i) for i in range(hourss)] + \
            ['{}_date_begin'.format(col), '{}_date_end'.format(col)]
        cols += i

    df = pd.DataFrame(
        data = a,
        columns=cols
    )
    
    return df

In [17]:
df = get_preliminare_stat(attrs)

ciscoRouter
ciscoSwitch
dslam
juniRouter
qtechRouter


In [18]:
for col in df.columns:
    if col not in ['model', 'ip'] and 'date' not in col:
        df[col] = df[col].astype(float)

In [19]:
df.head()

,model,ip,cpu_week_min_0,cpu_week_min_1,cpu_week_min_2,cpu_week_min_3,cpu_week_min_4,cpu_week_max_0,cpu_week_max_1,cpu_week_max_2,cpu_week_max_3,cpu_week_max_4,cpu_week_mean_0,cpu_week_mean_1,cpu_week_mean_2,cpu_week_mean_3,cpu_week_mean_4,cpu_week_median_0,cpu_week_median_1,cpu_week_median_2,cpu_week_median_3,cpu_week_median_4,cpu_week_std_0,cpu_week_std_1,cpu_week_std_2,cpu_week_std_3,cpu_week_std_4,cpu_week_q25_0,cpu_week_q25_1,cpu_week_q25_2,cpu_week_q25_3,cpu_week_q25_4,cpu_week_q75_0,cpu_week_q75_1,cpu_week_q75_2,cpu_week_q75_3,cpu_week_q75_4,cpu_week_integ_0,cpu_week_integ_1,cpu_week_integ_2,cpu_week_integ_3,cpu_week_integ_4,cpu_days_min_0,cpu_days_min_1,cpu_days_min_2,cpu_days_min_3,cpu_days_min_4,cpu_days_min_5,cpu_days_min_6,cpu_days_max_0,cpu_days_max_1,cpu_days_max_2,cpu_days_max_3,cpu_days_max_4,cpu_days_max_5,cpu_days_max_6,cpu_days_mean_0,cpu_days_mean_1,cpu_days_mean_2,cpu_days_mean_3,cpu_days_mean_4,cpu_days_mean_5,cpu_days_mean_6,cpu_days_median_0,cpu_days_median_1,cpu_days_median_2,cpu_days_median_3,cpu_days_median_4,cpu_days_median_5,cpu_days_median_6,cpu_days_std_0,cpu_days_std_1,cpu_days_std_2,cpu_days_std_3,cpu_days_std_4,cpu_days_std_5,cpu_days_std_6,cpu_days_q25_0,cpu_days_q25_1,cpu_days_q25_2,cpu_days_q25_3,cpu_days_q25_4,cpu_days_q25_5,cpu_days_q25_6,cpu_days_q75_0,cpu_days_q75_1,cpu_days_q75_2,cpu_days_q75_3,cpu_days_q75_4,cpu_days_q75_5,cpu_days_q75_6,cpu_days_integ_0,cpu_days_integ_1,cpu_days_integ_2,cpu_days_integ_3,cpu_days_integ_4,cpu_days_integ_5,cpu_days_integ_6,cpu_hours_min_0,cpu_hours_min_1,cpu_hours_min_2,cpu_hours_min_3,cpu_hours_min_4,cpu_hours_min_5,cpu_hours_min_6,cpu_hours_min_7,cpu_hours_min_8,cpu_hours_min_9,cpu_hours_min_10,cpu_hours_min_11,cpu_hours_min_12,cpu_hours_min_13,cpu_hours_min_14,cpu_hours_min_15,cpu_hours_min_16,cpu_hours_min_17,cpu_hours_min_18,cpu_hours_min_19,cpu_hours_min_20,cpu_hours_min_21,cpu_hours_min_22,cpu_hours_min_23,cpu_hours_max_0,cpu_hours_max_1,cpu_hours_max_2,cpu_hours_max_3,cpu_hours_max_4,cpu_hours_max_5,cpu_hours_max_6,cpu_hours_max_7,cpu_hours_max_8,cpu_hours_max_9,cpu_hours_max_10,cpu_hours_max_11,cpu_hours_max_12,cpu_hours_max_13,cpu_hours_max_14,cpu_hours_max_15,cpu_hours_max_16,cpu_hours_max_17,cpu_hours_max_18,cpu_hours_max_19,cpu_hours_max_20,cpu_hours_max_21,cpu_hours_max_22,cpu_hours_max_23,cpu_hours_mean_0,cpu_hours_mean_1,cpu_hours_mean_2,cpu_hours_mean_3,cpu_hours_mean_4,cpu_hours_mean_5,cpu_hours_mean_6,cpu_hours_mean_7,cpu_hours_mean_8,cpu_hours_mean_9,cpu_hours_mean_10,cpu_hours_mean_11,cpu_hours_mean_12,cpu_hours_mean_13,cpu_hours_mean_14,cpu_hours_mean_15,cpu_hours_mean_16,cpu_hours_mean_17,cpu_hours_mean_18,cpu_hours_mean_19,cpu_hours_mean_20,cpu_hours_mean_21,cpu_hours_mean_22,cpu_hours_mean_23,cpu_hours_median_0,cpu_hours_median_1,cpu_hours_median_2,cpu_hours_median_3,cpu_hours_median_4,cpu_hours_median_5,cpu_hours_median_6,cpu_hours_median_7,cpu_hours_median_8,cpu_hours_median_9,cpu_hours_median_10,cpu_hours_median_11,cpu_hours_median_12,cpu_hours_median_13,cpu_hours_median_14,cpu_hours_median_15,cpu_hours_median_16,cpu_hours_median_17,cpu_hours_median_18,cpu_hours_median_19,cpu_hours_median_20,cpu_hours_median_21,cpu_hours_median_22,cpu_hours_median_23,cpu_hours_std_0,cpu_hours_std_1,cpu_hours_std_2,cpu_hours_std_3,cpu_hours_std_4,cpu_hours_std_5,cpu_hours_std_6,cpu_hours_std_7,cpu_hours_std_8,cpu_hours_std_9,cpu_hours_std_10,cpu_hours_std_11,cpu_hours_std_12,cpu_hours_std_13,cpu_hours_std_14,cpu_hours_std_15,cpu_hours_std_16,cpu_hours_std_17,cpu_hours_std_18,cpu_hours_std_19,cpu_hours_std_20,cpu_hours_std_21,cpu_hours_std_22,cpu_hours_std_23,cpu_hours_q25_0,cpu_hours_q25_1,cpu_hours_q25_2,cpu_hours_q25_3,cpu_hours_q25_4,cpu_hours_q25_5,cpu_hours_q25_6,cpu_hours_q25_7,cpu_hours_q25_8,cpu_hours_q25_9,cpu_hours_q25_10,cpu_hours_q25_11,cpu_hours_q25_12,cpu_hours_q25_13,cpu_hours_q25_14,cpu_hours_q25_15,cpu_hours_q25_16,cpu_hours_q25_17,cpu_hours_q25_18,cpu_hours_q25_19,cpu_hours_q25_20,cpu_hours_q25_21,cpu_hours_q25_22,cpu_hours_q

## windows clustering

Если взять все устройства. Каждый временной ряд по определенному параметру нарезать на окна, нормализовать и попробовать кластеризовать. Что - нибудь получится?

In [20]:
def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

In [21]:
from sklearn.preprocessing import Normalizer

In [29]:
norm = Normalizer()
ts_windows = None
i = 0
j = 0
s = 0
for model in df['model'].unique():
    sample = df[df['model'] == model].sample(10)
    print(sample.shape)
    for ix, row in sample.iterrows():
        try:
            ts = attrs[model][row.ip]['inUtilization']['ts']
        except:
            s += 1
        if isinstance(ts, int):
            j += 1
            continue
        i +=1 
        windows = rolling_window(ts, 12)
        norm_windows = norm.fit_transform(windows)
        if ts_windows is None:
            ts_windows = norm_windows
        else:
            ts_windows = np.vstack([ts_windows, norm_windows])

(10, 1162)
(10, 1162)
(10, 1162)
(10, 1162)
(10, 1162)


In [30]:
ts_windows.shape

(1315552, 12)

In [31]:
tsne = TSNE(n_components=2, perplexity=35)

In [ ]:
X = tsne.fit_transform(ts_windows)

In [ ]:
sns.lm

In [ ]:
# df.select_dtypes(include=['float']).hist(bins=100);

In [ ]:
# df.columns.tolist()

## wavelet transformation

In [ ]:
def wt_presentation(df, attrs):
    n = 10
    dr = df.copy(deep=True)
    r = []
    for i in ['out', 'in']:
        r += [
            '{}_coeff_{}'.format(i,ii)
            for ii in range(1, n)
        ]

    d = pd.DataFrame(data = np.zeros((df.shape[0], len(r))), columns=r)
        
    for ix, row in dr.iterrows():
        # dict_keys(['cpuUtil', 'memUtil', 'inUtilization', 'outUtilization'])
        # attrs['ciscoSwitch']['10.10.115.116']['inUtilization'].keys()
        # dict_keys(['df', 'ts', 'coeff', 'freq'])
        for k, v in attrs[row.model][row.ip].items():
            for i in ['out', 'in']:
                if i in k.lower():
                    coeff = v['coeff']
                    freq = v['freq']
                    for r in range(n-1):
                        if not isinstance(coeff, int):
                            ixs = heapq.nlargest(1,range(coeff.shape[1]), coeff[r,:].take)[0]
                            d.iloc[ix]['{}_coeff_{}'.format(i,r+1)] = coeff[r, ixs]
                        else:
                            d.iloc[ix]['{}_coeff_{}'.format(i,r+1)] = 0
                    break
                    
    return d

In [ ]:
def concat_df_and_wt(df, wt):
    dd = pd.concat([df, wt], axis=1)
    scaler = MinMaxScaler()
    rr = scaler.fit_transform(dd)
    return pd.DataFrame(data=rr, columns=dd.columns.tolist())

In [ ]:
q = wt_presentation(df, attrs)

In [ ]:
col_without_date = [col for col in df.columns if 'date' not in col and col not in ['ip', 'model']]

In [ ]:
dd = df[col_without_date]

In [ ]:
df_and_wt = concat_df_and_wt(dd, q)

In [ ]:
X = df_and_wt.values

In [ ]:
X.shape

# PAA

In [ ]:
from sklearn.preprocessing import normalize

In [ ]:
def paa(ts, w):
    n = ts.shape[0]
    ts = (ts-np.mean(ts))/np.std(ts)
    paa_repres = [w/n * np.sum(ts[int(n/w)*(i-1):int(n/w)*(i)]) for i in range(1, w+1)]
    return ts, np.asarray(paa_repres)

In [ ]:
def paa_visualisation(ts, paa_repres):
    paa_vis = []
    w = paa_repres.shape[0]
    n = ts.shape[0]
    k = 0
    j = 0
    for  i in range(n):
        if  j < int(n/w):
            paa_vis.append(paa_repres[k])
            j += 1
        else:
            j = 0
            k += 1
            paa_vis.append(paa_repres[k])
    return np.asarray(paa_vis)

In [ ]:
ts = attrs['ciscoRouter']['62.183.0.103']['outUtilization']['ts']

In [ ]:
ts.shape[0]/12

In [ ]:
chain = 12*2

In [ ]:
norm_ts, paa_repres = paa(ts.values[:int(ts.shape[0]/chain)*chain], w=int(ts.shape[0]/chain))

In [ ]:
paa_vis = paa_visualisation(norm_ts, paa_repres)

In [ ]:
from scipy.stats import probplot

In [ ]:
probplot(norm_ts, plot=plt);

In [ ]:
# plt.figure(figsize=(15,9))
# plt.plot(norm_ts[:paa_vis.shape[0]], c='g')
# plt.plot(paa_vis[:paa_vis.shape[0]], c='r')

In [ ]:
def sax_mapper(paa_repres, coef_count):
    sym = list('abcdefghjk')
    coeff_table = {
        3:[-0.43, 0.43],
        4:[-0.67, 0, 0.67],
        5:[-0.84, -0.25, 0.25, 0.84],
        6:[-0.97, -0.43, 0, 0.43, 0.97],
        7:[-1.07, -0.57, -0.18, 0.18, 0.57, 1.07],
        8:[-1.15, -0.67, -0.32, 0, 0.32, 0.67, 1.15],
        9:[-1.22, -0.76, -0.43, -0.14, 0.14, 0.43, 0.76, 1.22],
        10:[-1.28, -0.84, -0.52, -0.25, 0, 0.25, 0.52, 0.84, 1.28]
    }
    unique = np.unique(paa_repres)
    work_sym = sym[:coef_count]
    work_coeffs = coeff_table.get(coef_count, -1)
    if work_coeffs == -1:
        raise Exception('choose valid number of coefficients')
    tsvalue_2_sym = {}
    work_coeffs = [-np.inf] + work_coeffs + [np.inf]

    
    for elem in unique:
        was = False
        for ix in range(1, len(work_coeffs)):
            if work_coeffs[ix-1] <= elem < work_coeffs[ix]:
                was = True
                w = work_sym[ix]
                tsvalue_2_sym[elem] = w
                break
        
        if not was:
            print(elem)
        
    return tsvalue_2_sym

In [ ]:
tsvalue_2_sym = sax_mapper(paa_vis, 10)

In [ ]:
np.unique(list(tsvalue_2_sym.values()))

In [ ]:
y1 = norm_ts[:900]
y2 = paa_vis[:900]

In [ ]:
unique_marker = list(np.unique(list(tsvalue_2_sym.values())))

In [ ]:
color_mapper = {}
for sym in list(set('abcdefghjk')):
    color_mapper[sym] = np.random.rand(3,1).flatten()

In [ ]:
plt.figure(figsize=(15,9))
plt.plot(y1)
for i in range(y2.shape[0]):
#     print(color_mapper[tsvalue_2_sym[y2[i]]])
    plt.scatter(i, y2[i], color=color_mapper[tsvalue_2_sym[y2[i]]])

# SOM

In [ ]:
import numpy as np
import tensorflow as tf

class SOMNetwork():
    def __init__(self, input_dim, dim=10, sigma=None, learning_rate=0.1, tay2=1000, dtype=tf.float32):
        #если сигма на определена устанавливаем ее равной половине размера решетки
        if not sigma:
            sigma = dim / 2
        self.dtype = dtype
        #определяем константы использующиеся при обучении
        self.dim = tf.constant(dim, dtype=tf.int64)
        self.learning_rate = tf.constant(learning_rate, dtype=dtype, name='learning_rate')
        self.sigma = tf.constant(sigma, dtype=dtype, name='sigma')
        #тау 1 (формула 6)
        self.tay1 = tf.constant(1000/np.log(sigma), dtype=dtype, name='tay1')
        #минимальное значение сигма на шаге 1000 (определяем по формуле 3)
        self.minsigma = tf.constant(sigma * np.exp(-1000/(1000/np.log(sigma))), dtype=dtype, name='min_sigma')
        self.tay2 = tf.constant(tay2, dtype=dtype, name='tay2')
        #input vector
        self.x = tf.placeholder(shape=[input_dim], dtype=dtype, name='input')
        #iteration number
        self.n = tf.placeholder(dtype=dtype, name='iteration')
        #матрица синаптических весов
        self.w = tf.Variable(tf.random_uniform([dim*dim, input_dim], minval=-1, maxval=1, dtype=dtype),
            dtype=dtype, name='weights')
        #матрица позиций всех нейронов, для определения латерального расстояния
        self.positions = tf.where(tf.fill([dim, dim], True))
        
    def __competition(self, info=''):
        with tf.name_scope(info+'competition') as scope:
            #вычисляем минимум евклидова расстояния для всей сетки нейронов
            distance = tf.sqrt(tf.reduce_sum(tf.square(self.x - self.w), axis=1))
        #возвращаем индекс победившего нейрона (формула 1)
        return tf.argmin(distance, axis=0)
    
    def feed(self, input):
        init = tf.global_variables_initializer()
        with tf.Session() as sess:
            init.run()
            win_index = sess.run(self.__competition(), feed_dict={self.x: input})
            win_index_2d = np.array([win_index//self.dim.eval(), win_index-win_index//self.dim.eval()*self.dim.eval()])
        return win_index_2d
    
    def training_op(self):
       #определяем индекс победившего нейрона
        win_index = self.__competition('train_')
        with tf.name_scope('cooperation') as scope:
            #вычисляем латеральное расстояние d
            #для этого переводим инедкс победившего нейрона из 1d координаты в 2d координату
            coop_dist = tf.sqrt(tf.reduce_sum(tf.square(tf.cast(self.positions -
                [win_index//self.dim, win_index-win_index//self.dim*self.dim], 
                dtype=self.dtype)), axis=1))
            #корректируем сигма (используя формулу 3)
            sigma = tf.cond(self.n > 1000, lambda: self.minsigma, lambda: self.sigma * tf.exp(-self.n/self.tay1))
            #вычисляем топологическую окрестность (формула 2)
            tnh = tf.exp(-tf.square(coop_dist) / (2 * tf.square(sigma)))
        with tf.name_scope('adaptation') as scope:
            #обновляем параметр скорости обучения (формула 5)
            lr = self.learning_rate * tf.exp(-self.n/self.tay2)
            minlr = tf.constant(0.01, dtype=self.dtype, name='min_learning_rate')
            lr = tf.cond(lr <= minlr, lambda: minlr, lambda: lr)
            #вычисляем дельта весов и обновляем всю матрицу весов (формула 4)
            delta = tf.transpose(lr * tnh * tf.transpose(self.x - self.w))
            training_op = tf.assign(self.w, self.w + delta)
        return training_op

In [ ]:
som = SOMNetwork(input_dim=242, dim=20, dtype=tf.float64, sigma=3)

In [ ]:
# training_op = som.training_op()
# init = tf.global_variables_initializer()
# with tf.Session() as sess:
#     init.run()
#     for i, color_data in enumerate(X):
#         if i % 1000 == 0:
#             print('iter:', i)
#         sess.run(training_op, feed_dict={som.x: color_data, som.n:i})

In [ ]:
type(som.x)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, X, test_size=0.1, random_state=42)

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]))
X_test  = np.reshape(X_test,  (X_test.shape[0], X_test.shape[1]))

In [ ]:
X_train.shape

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# AUTOENCODER

# VAE

In [ ]:
from keras import backend as K
from keras import objectives

In [ ]:
batch_size = 256
original_dim = X.shape[1]
latent_dim = 3
intermediate_dim = 256
nb_epoch = 50
epsilon_std = 1.0

z_mean = None
z_log_var = None
z = None

In [ ]:
K.random_normal?

In [ ]:
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(latent_dim, ), mean=0.,
                              stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

In [ ]:
def vae_loss(x, x_decoded_mean):
    global z_mean, z_log_var, z
#     print(np.asarray(x_decoded_mean[:]))
    xent_loss = original_dim *  objectives.mean_squared_error(x, x_decoded_mean)
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return xent_loss + kl_loss

In [ ]:
def create_vae():
    global z_mean, z_log_var, z
    x = Input(shape=(original_dim,))
    encoded = Dense(intermediate_dim*2, activation='relu')(x)
    encoded = Dense(intermediate_dim, activation='relu')(encoded)
    encoded = Dense(int(intermediate_dim/2), activation='relu')(encoded)

    z_mean = Dense(latent_dim)(encoded)
    z_log_var = Dense(latent_dim)(encoded)
    z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

    h_decoded = Dense(int(intermediate_dim/2), activation='relu')(z)
    h_decoded = Dense(intermediate_dim, activation='relu')(h_decoded)
    h_decoded = Dense(intermediate_dim*2, activation='relu')(h_decoded)
    x_decoded_mean = Dense(original_dim, activation='relu')(h_decoded)
    
    vae = Model(inputs=x, outputs=x_decoded_mean)
    vae.compile(optimizer='adam', loss=vae_loss)
    
    encoder = Model(x, z_mean)
    
    
    return vae, encoder

In [ ]:
vae, encoder = create_vae()

In [ ]:
vae.summary()

In [ ]:
## train the VAE on MNIST digits
vae.fit(X_train, X_train,
        shuffle=True,
        nb_epoch=500,
        batch_size=batch_size,
        validation_data=(X_test, X_test))

# ## display a 2D plot of the digit classes in the latent space
# x_test_encoded = encoder.predict(X_test, batch_size=batch_size)
# plt.figure(figsize=(6, 6))
# plt.scatter(x_test_encoded[:, 0], x_test_encoded[:, 1])#, c=y_test)
# plt.colorbar()
# plt.show()

In [ ]:
## display a 2D plot of the digit classes in the latent space
x_test_encoded = encoder.predict(X, batch_size=batch_size)
plt.figure(figsize=(6, 6))
plt.scatter(x_test_encoded[:, 2], x_test_encoded[:, 1])#, c=y_test)
# plt.colorbar()
plt.show()

## AE

In [ ]:
from keras.layers import Input, Dense, Flatten, Reshape, Lambda
from keras.models import Model

def create_dense_ae(input_dim, encoding_dim):

    input_x = Input(shape=(input_dim,)) 
    encoded = Dense(128, activation='relu')(input_x)
    encoded = Dense(64, activation='relu')(encoded)
    encoded = Dense(32, activation='relu')(encoded)
    encoded = Dense(encoding_dim, activation='relu')(encoded)
    
    input_encoded = Input(shape=(encoding_dim,))
    decoded = Dense(32, activation='relu')(input_encoded)
    decoded = Dense(64, activation='relu')(decoded)
    decoded = Dense(128, activation='relu')(decoded)
    decoded = Dense(input_dim, activation='sigmoid')(decoded)
    
    encoder = Model(input_x, encoded, name="encoder")
    decoder = Model(input_encoded, decoded, name="decoder")
    autoencoder = Model(input_x, decoder(encoder(input_x)), name="autoencoder")
    return encoder, decoder, autoencoder

In [ ]:
encoder, decoder, autoencoder = create_dense_ae(input_dim=X_train.shape[1],encoding_dim=10)
autoencoder.compile(optimizer='adam', loss='mse')

In [ ]:
autoencoder.summary()

In [ ]:
%%time
autoencoder.fit(X_train, X_train,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test, X_test))

In [ ]:
encoded = encoder.predict(X)
encoded[0]

In [ ]:
sns.pairplot(pd.DataFrame(encoded))

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
# for i in [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]:
#     tsne = TSNE(n_components=2, perplexity=i)
#     res_X = tsne.fit_transform(encoded)
    
#     d = pd.DataFrame()
#     d['x'] = res_X[:,0]
#     d['y'] = res_X[:,1]
    
#     plt.figure()
#     sns.lmplot(data=d, x='x', y='y', fit_reg=False, size=10, aspect=1.2)

In [ ]:
tsne = TSNE(n_components=2, perplexity=30)
res_X = tsne.fit_transform(encoded)

d = pd.DataFrame()
d['x'] = res_X[:,0]
d['y'] = res_X[:,1]

plt.figure()
sns.lmplot(data=d, x='x', y='y', fit_reg=False, size=10, aspect=1.2)

In [ ]:
d = pd.DataFrame()
d['x'] = res_X[:,0]
d['y'] = res_X[:,1]
d['cls'] = df.model.values

plt.figure()
sns.lmplot(data=d, x='x', y='y', hue='cls', fit_reg=False, size=10, aspect=1.2)

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage

In [ ]:
Z = linkage(res_X, 'ward')

In [ ]:
plt.figure(figsize=(15,9))
dendrogram(
    Z,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=2.,  # font size for the x axis labels
);

In [ ]:
%matplotlib inline

# %matplotlib notebook
fig = plt.figure(figsize=(17,10))
ax = fig.gca(projection='3d')

# rng = np.arange(0,300)
# X, Y = np.meshgrid(encoded_X['Z'].values, encoded_X['Y'].values)
# Z = encoded_X['Q'].values

# surf = ax.scatter(encoded_X['Z'],encoded_X['Y'],Z, cmap='viridis',linewidth=0)
surf = ax.scatter(d['x'].values, d['y'].values, d['z'].values)

# Customize the z axis.
# ax.set_zlim(-4.001, 4.001)
# ax.set_xlim(rng[0], rng[-1])
# # ax.set_ylim(0, 59)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# common cluster builder function

In [ ]:
def common_cluster_builder(df):
    scaler = MinMaxScaler()
    aa = df.select_dtypes(include='float64')
    rr = scaler.fit_transform(aa.fillna(-1))
    he = df.iloc[:,:2]
    xx = pd.DataFrame(data=rr, columns=aa.columns)
    fd = pd.concat([he, xx], axis=1, ignore_index=True)
    fd.columns = he.columns[:].values.tolist() + xx.columns[:].values.tolist()
    return fd

# clusters with model

In [ ]:
def clusters_with_model(df):
    fd = common_cluster_builder(df)
    X = pd.get_dummies(fd, columns=['model'])
    ip = X['ip']
    model = df['model']
    X.drop(['ip'], axis=1, inplace=True)
    return X, ip, model

# clusters with pywt and model

In [ ]:
def clusters_with_wt_and_model(df):
    X, ip, model = clusters_with_model(df)
    return X, ip, model

# clusters without model

In [ ]:
def clusters_without_model(df):
    fd = common_cluster_builder(df)
    ip = fd['ip']
    model = fd['model']
    fd.drop(['ip','model'], axis=1, inplace=True)
    return fd, ip, model   

# clusters with pywt and no model

In [ ]:
def cluster_with_wt_and_no_model(df):
    return clusters_without_model(df)

### get data

In [ ]:
q = wt_presentation(df, attrs)

In [ ]:
X_with_model, ip, model = clusters_with_model(df)

In [ ]:
XX, ip, model = clusters_with_wt_and_model(df)
X_with_wt_and_model = concat_df_and_wt(XX, q)

In [ ]:
X_without_model, ip, model = clusters_without_model(df)

In [ ]:
XX, ip, model = cluster_with_wt_and_no_model(df)
X_with_wt_and_withou_model = concat_df_and_wt(XX, q)

In [ ]:
X_with_wt_and_withou_model.describe()

# transformation

In [ ]:
def get_transformed_data(X):
    tsne = TSNE(n_components=2)
    pca = PCA(n_components=2)
    X_prep_tsne = tsne.fit_transform(X)
    X_prep_pca = pca.fit_transform(X)
    return X_prep_tsne, X_prep_pca

In [ ]:
def plot_scatter(X):
    f = plt.figure(figsize=(19,9))
    plt.scatter(X[:,0], X[:,1])
    return f

In [ ]:
def plot_with_model(X,model):
    X1_prep_with_model = pd.DataFrame()
    X1_prep_with_model['X'] = X[:,0]
    X1_prep_with_model['Y'] = X[:,1]
    X1_prep_with_model['model'] = model.values
    f = sns.lmplot(data=X1_prep_with_model, x='X', y='Y', hue='model', fit_reg=False, size=10, aspect=1.2);
    return f

In [ ]:
def get_data_and_plots(df):
    X_prep_tsne, X_prep_pca = get_transformed_data(df)
    f1 = plot_scatter(X_prep_tsne)
    f2 = plot_scatter(X_prep_pca)
    f3 = plot_with_model(X_prep_tsne, model)
    return X_prep_tsne, X_prep_pca

In [ ]:
X_tsne_model, X_pca_model = get_data_and_plots(X_with_model);

In [ ]:
X_tsne_model_wt, X_pca_model_wt = get_data_and_plots(X_with_wt_and_model);

In [ ]:
X_tsne, X_pca = get_data_and_plots(X_without_model);

In [ ]:
X_tsne_wt, X_pca_wt = get_data_and_plots(X_with_wt_and_withou_model);

In [ ]:
X_tsne_model_wt

X1_prep_with_model = pd.DataFrame()
X1_prep_with_model['X'] = X_tsne_model_wt[:,0]
X1_prep_with_model['Y'] = X_tsne_model_wt[:,1]
X1_prep_with_model['model'] = model.values
f = sns.lmplot(data=X1_prep_with_model, x='X', y='Y', hue='model', fit_reg=False, size=10, aspect=1.2);

In [ ]:
attrs['ciscoRouter']['62.183.0.119']['inUtilization']['ts']

In [ ]:
model.unique()

In [ ]:
r = 1
plt.figure(figsize=(15,9))
for i, mod in enumerate(model.unique()):
    print(mod)
    ips = np.asarray(list(attrs[mod].keys()))
    for ix, ip in enumerate(np.random.choice(ips, 4)):
        print(ip)
        if not isinstance(attrs[mod][ip]['inUtilization']['ts'], int):
            ts = attrs[mod][ip]['inUtilization']['ts'].values
        else:
            j = True
            while j:
                ipp = np.random.choice(ips)
                if not isinstance(attrs[mod][ipp]['inUtilization']['ts'], int):
                    ts = attrs[mod][ipp]['inUtilization']['ts'].values
                    j = False
        print('{} {} {}'.format(5,4, r))
        plt.subplot(5,4, r)
        plt.plot(range(ts.shape[0])[:150], ts[:150])
        r += 1
    

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
def get_bgm_clastering(X, n_components, max_iter):
    bgm = BayesianGaussianMixture(
        n_components=n_components, 
        max_iter=max_iter, 
        init_params='kmeans', 
        verbose=True,
        tol=1e-5
    )
    
    bgm = DBSCAN(n_jobs=-1)
    
    df = pd.DataFrame()
    df['X'] = X[:,0]
    df['Y'] = X[:,1]
    bgm.fit(X)
    
    another_X = pd.DataFrame()
    another_X['X'] = X[:,0]
    another_X['Y'] = X[:,1]
    another_X['cls'] =  bgm.fit_predict(X)
    
    return another_X

In [ ]:
another_X = get_bgm_clastering(res_X, 25, 10000)

In [ ]:
plt.vlines?

In [ ]:
sns.lmplot(data=another_X, x='X', y='Y', fit_reg=False, size=10, aspect=1.2);
plt.xticks(range(int(res_X[:,0].min()),int(res_X[:,0].max())), rotation=90)
plt.yticks(range(int(res_X[:,1].min()),int(res_X[:,1].max())))
plt.hlines(-14, -50, 50)
plt.hlines(-20, -50, 50)
plt.vlines(-35, -50, 50)
plt.vlines(-29, -50, 50)
# plt.plot([-29, -38, -32, -26, -29], [-22, -15, -10, -18, -22], c='y')

In [ ]:
mask1 = (res_X[:,1] < -14) & (res_X[:,1] > -20)
mask2 = (res_X[:,0] < -29) & (res_X[:,0] > -35)

In [ ]:
xx = res_X[mask1 & mask2]

In [ ]:
xx.shape

In [ ]:
fx = pd.DataFrame()
fx['x'] = xx[:,0]
fx['y'] = xx[:,1]
fx['cls'] = ['{}!{}'.format(model, ip) for model,ip in zip(aa.model.values, aa.ip.values)]

In [ ]:
aa.shape

In [ ]:
sns.lmplot(data=fx, x='x', y='y', hue='cls', fit_reg=False, size=10, aspect=1.2);

In [ ]:
res = another_X[(another_X['X'].isin(xx[:,0])) & (another_X['Y'].isin(xx[:,1]))]

In [ ]:
aa = df[df.index.isin(res.index)][['model','ip']]

In [ ]:
for ix, row in aa.iterrows():
    tt = attrs[row.model][row.ip]
    try:
        plt.figure()
        tt['outUtilization']['ts'].plot()
        plt.title('{} and {}'.format(row.model, row.ip))
    except:
        print('ERROR {} and {}'.format(row.model, row.ip))
        

    

In [ ]:
sns.pairplot(df[df.index.isin(aa.index)])

In [ ]:
df[df.index.isin(aa.index)].T

In [ ]:
tt['outUtilization']['ts'].head()

In [ ]:
# for i in range(10,40):
#     X = get_bgm_clastering(X_tsne_wt, i, 5000)
#     sns.lmplot(data=X, x='X', y='Y', hue='cls', fit_reg=False, size=10, aspect=1.2)
#     plt.title(i)

In [ ]:
X = get_bgm_clastering(X_tsne_wt, 3, 5000)
sns.lmplot(data=X, x='X', y='Y', hue='cls', fit_reg=False, size=10, aspect=1.2);

In [ ]:
X.cls.unique()

In [ ]:
res.to_csv('./cluster_with_noMem_and_noModel_but_with_wt.csv', sep=';', encoding='utf-8')

In [ ]:
dd = df[X.cls==1]

In [ ]:
np.random.seed(7)

In [ ]:
dd.head()

In [ ]:
dd.model.value_counts()

In [ ]:
def prepare_data_for_LSTM(y, look_back=1):
    def create_dataset(dataset, look_back=1):
        dataX, dataY = [], []
        for i in range(len(dataset)-look_back-1):
            a = dataset[i:(i+look_back), 0]
            dataX.append(a)
            dataY.append(dataset[i + look_back, 0])
        return np.array(dataX), np.array(dataY)
    
    dataset = y.astype('float32')
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset.reshape(-1,1))
    
    train_size = int(len(dataset) * 0.67)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
    print(len(train), len(test))
    
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
    
    return trainX, trainY, testX, testY

In [ ]:
def create_LSTM(trainX, trainY, epochs=1, look_back=1):
    model = Sequential()
    model.add(LSTM(2, input_shape=(1, look_back)))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(trainX, trainY, epochs=epochs, batch_size=1, verbose=1)
    return model

Попробовать следующее:
Посмотреть на интервалы времени. выбрал пересечение этих интервалов. Найти среднее за каждую отметку по кластеру и обучить LSTM на этом. Затем взять рандомных 5 устройств из кластера и посмотреть на качество.

Если интервал времени будет маленький, то хз что делать.
Можно например взять большой интервал. И на пересечении искать среднее, а слева и справа заменять существующими значениями нормированными.

In [ ]:
def get_resampled_ts(ts):
    indx = ts.resample('5T').index.values
    a = ts.resample('5T').reset_index(drop=True).interpolate(method='linear')
    return pd.Series(data=a.values, index=indx)

In [ ]:
def get_max_interval(X, data, attrs, cluster, typo):
    glob_max_date = None
    glob_min_date = None
    df = data[X.cls==cluster]
    for mod in df['model'].unique():
        for ip in df['ip'].unique():
            dd = attrs[mod][ip][typo]['ts']
            resampled_ts = get_resampled_ts(dd)
            glob_max_date = np.max(resampled_ts.index.values) if glob_max_date is None or glob_max_date < np.max(resampled_ts.index.values) else glob_max_date
            glob_min_date = np.min(resampled_ts.index.values) if glob_min_date is None or glob_min_date > np.min(resampled_ts.index.values) else glob_min_date   
            
    glob_max_date = pd.DatetimeIndex([glob_max_date])[0]
    glob_min_date = pd.DatetimeIndex([glob_min_date])[0]
    glob_min_date = dtt.datetime(
        glob_min_date.year,
        glob_min_date.month,
        glob_min_date.day,
        glob_min_date.hour,
        glob_min_date.minute,
        glob_min_date.second
    )
    glob_max_date = dtt.datetime(
        glob_max_date.year,
        glob_max_date.month,
        glob_max_date.day,
        glob_max_date.hour,
        glob_max_date.minute,
        glob_max_date.second
    )
    return glob_min_date, glob_max_date

In [ ]:
def get_full_separated_interval(begin, end):
    time = []
    start = dtt.datetime(begin.year,begin.month,begin.day,begin.hour,begin.minute,begin.second)
    while start < end:
        time.append(start)
        start += dtt.timedelta(minutes=5)
    return time

In [ ]:
def prep_data_for_clusterLSTM(X, data, attrs, cluster, typo, min_date, max_date):
    time = get_full_separated_interval(min_date, max_date)
    time_dict = {}
    df = data[X.cls==cluster]
    for mod in df['model'].unique():
        for ip in df['ip'].unique():
            ts = attrs[mod][ip][typo]['ts']
            resampled_ts = get_resampled_ts(ts)
#             return 1
            for t in time:
                if t in resampled_ts.index:
                    time_dict.setdefault(t, []).append(resampled_ts[t])
    return time_dict   

In [ ]:
attrs['ciscoRouter']['62.183.0.46'].keys()

In [ ]:
mask = df.model == 'dslam'

In [ ]:
min_date, max_date = get_max_interval(
    X=X[mask], 
    data=df[mask], 
    attrs=attrs, 
    cluster=1, 
    typo='inUtilization'
)

In [ ]:
min_date

In [ ]:
max_date

In [ ]:
time_dict = prep_data_for_clusterLSTM(    
    X=X[mask], 
    data=df[mask], 
    attrs=attrs, 
    cluster=1, 
    typo='inUtilization',
    min_date=min_date,
    max_date=max_date
)

In [ ]:
time_2_mean = {
    time: np.mean(np.asarray(vals))
    for time, vals in time_dict.items()
}

In [ ]:
for time, val in time_2_mean.items():
    print(val)
    break

In [ ]:
time_and_mean = pd.Series(time_2_mean,index=time_2_mean.keys())

In [ ]:
plt.figure(figsize=(19,9))
time_and_mean.plot()

In [ ]:
def predict_LSTM(y2, trainX2, trainY2, testX2, testY2, look_back = 1):
    dataset = y2.astype('float32')
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset.reshape(-1,1))

    trainPredict = lstm_model.predict(trainX2)
    testPredict = lstm_model.predict(testX2)

    # invert predictions
    trainPredict = scaler.inverse_transform(trainPredict)
    trainY2 = scaler.inverse_transform([trainY2])
    testPredict = scaler.inverse_transform(testPredict)
    testY2 = scaler.inverse_transform([testY2])


    # calculate root mean squared error
    trainScore = mean_squared_error(trainY2[0], trainPredict[:,0])
    print('Train Score: %.2f MSE' % (trainScore))
    testScore = mean_squared_error(testY2[0], testPredict[:,0])
    print('Test Score: %.2f MSE' % (testScore))
    
    
    trainPredictPlot = np.empty_like(dataset)
    trainPredictPlot[:, :] = np.nan
    trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
    # shift test predictions for plotting
    testPredictPlot = np.empty_like(dataset)
    testPredictPlot[:, :] = np.nan
    trainPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
    
    # plot baseline and predictions
    plt.figure(figsize=(15,9))
    plt.title('Графики действительного и предсказанного c помощью LSTM значения inUtil')
    plt.xlabel('time')
    plt.ylabel('inUtil')
    plt.plot(scaler.inverse_transform(dataset), label='Действительные значения')
    plt.plot(trainPredictPlot, label='Предсказанные значения')
    plt.legend()
    
    # plot baseline and predictions
    plt.figure(figsize=(15,9))
    plt.title('Увеличенный масштаб графиков действительного и предсказанного c помощью LSTM значения inUtil')
    plt.xlabel('time')
    plt.ylabel('inUtil')
    plt.plot(scaler.inverse_transform(dataset)[:300], label='Действительные значения')
    plt.plot(trainPredictPlot[:300], label='Предсказанные значения')
    plt.legend()

In [ ]:
y1 = time_and_mean.values

In [ ]:
trainX1, trainY1, testX1, testY1 = prepare_data_for_LSTM(y1)

In [ ]:
lstm_model = create_LSTM(trainX1, trainY1)

In [ ]:
predict_LSTM(y1, trainX1, trainY1, testX1, testY1, look_back = 1)

In [ ]:
y2 = attrs['dslam']['10.10.104.105']['inUtilization']['ts'].values

In [ ]:
trainX2, trainY2, testX2, testY2 = prepare_data_for_LSTM(y2)

In [ ]:
predict_LSTM(y2, trainX2, trainY2, testX2, testY2, look_back = 1)

In [ ]:
y3 = attrs['dslam']['10.10.142.63']['inUtilization']['ts'].values

In [ ]:
trainX3, trainY3, testX3, testY3 = prepare_data_for_LSTM(y3)

In [ ]:
predict_LSTM(y3, trainX3, trainY3, testX3, testY3, look_back = 1)

# Motifs

In [ ]:
def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

In [ ]:
def movstd(ts, window_size):
    return np.std(rolling_window(ts, window_size), axis=1)

def mass(ts, q, mstd, eps=1e-8):
    window_size = q.shape[0]
    dots = scipy.signal.fftconvolve(ts, q[::-1], mode='valid')
    dist = np.sqrt(np.abs(2 * (window_size - np.divide(dots, mstd))))
    return dist

def stamp(ts, windows, mode='full'):
    window_size = windows.shape[1]
    windows_count = windows.shape[0]
    matrix_profile = np.zeros(windows_count) + np.inf
    matrix_profile_index = -np.ones_like(matrix_profile, dtype=np.int32)
    mstd = movstd(ts, window_size)
    
    if type(mode) is str and mode == 'full':
        end_value = windows_count
    elif type(mode) is float:
        end_value = int(mode * windows_count)
    elif type(mode) is int:
        end_value = mode
    else:
        raise ValueError
    
    for i in tqdm.tqdm_notebook(np.random.permutation(windows_count)[:end_value]):
        d = mass(ts, windows[i], mstd)
        d[max(int(i - window_size / 4), 0):min(int(i + window_size / 4 - 1), windows_count)] = np.inf
        matrix_profile_index[np.where(d <= matrix_profile)] = i
        matrix_profile = np.minimum(matrix_profile, d)
        
    return matrix_profile, matrix_profile_index

In [ ]:
def get_x_windows(ts, window_size=12):
    x_roll = rolling_window(ts, window_size).squeeze()
    xw_mean = np.mean(x_roll, axis=1)
    xw_std = np.std(x_roll, axis=1)
    x_windows = np.divide(x_roll - xw_mean[:, np.newaxis], xw_std[:, np.newaxis])

In [ ]:
def get_MP_and_MPI(ts, window_size=12):
    x_windows = get_x_windows(ts, window_size)
    MP, MPI = stamp(ts, x_windows, mode=5000)
    return MP, MPI

In [ ]:
def plot_example(index=300, ts, MP):
    i = 0
    plt.figure(figsize=(19,9))
    plt.scatter(range(ts[i:index].shape[0]), 3*ts.values[i:index]+3)
    plt.plot(range(MP[i:index].shape[0]), MP[i:index]);

In [ ]:
def get_nlargest_index(n = 10, dataset):
    return heapq.nlargest(n, range(len(dataset)), dataset.take)

In [ ]:
def get_nsmallest(n=10, dataset)
    return heapq.nsmallest(10,range(len(dataset)), dataset.take)

In [ ]:
df = X.copy(deep=True)

In [ ]:
dfs = {}
for cl in X.cls.unique():
    dfs[cl] = X[X.cls == cl][['X','Y']]

In [ ]:
dfs.keys()

In [ ]:
Y1 = get_bgm_clastering(dfs[5][['X','Y']].values, 3, 5000)
sns.lmplot(data=Y1, x='X', y='Y', hue='cls', fit_reg=False, size=10, aspect=1.2);

In [ ]:
Y2 = get_bgm_clastering(dfs[2][['X','Y']].values, 6, 5000)
sns.lmplot(data=Y2, x='X', y='Y', hue='cls', fit_reg=False, size=10, aspect=1.2);

In [ ]:
r = pd.DataFrame(columns=['X', 'Y', 'cls'])

In [ ]:
done_df = df[~df.cls.isin([2,5])]

In [ ]:
not_done_df = df[df.cls.isin([2,5])]

In [ ]:
done_df.cls.unique()

In [ ]:
Y1.cls.unique()

In [ ]:
Y1.cls = Y1.cls.map({2:20, 0:10, 1:15})

In [ ]:
Y1.cls.unique()

In [ ]:
Y2.cls.unique()

In [ ]:
Y2.cls = Y2.cls.map({0:30, 1:35, 2:40, 4:45, 5:50})

In [ ]:
f = pd.concat([done_df, Y1, Y2], axis=0, ignore_index=True)

In [ ]:
sns.lmplot(data=f, x='X', y='Y', hue='cls', fit_reg=False, size=10, aspect=1.2);

In [ ]:
f.cls.unique()

In [ ]:
we = df.copy(deep=True)

In [ ]:
we['model'] = model
we['ip'] = ip

In [ ]:
we.head()

In [ ]:
f['model'] = ''
f['ip'] = ''

In [ ]:
f.head()

In [ ]:
res = pd.merge(f, we, how='inner', on=['X','Y'])

In [ ]:
res = res[['model_y','ip_y','X','Y','cls_x']]

In [ ]:
res.columns = ['model','ip', 'X', 'Y', 'cls']

In [ ]:
res.head()

In [ ]:
sns.lmplot(data=res, x='X', y='Y', hue='cls', fit_reg=False, size=10, aspect=1.2);

In [ ]:
attrs['ciscoRouter']['62.183.0.103'].keys()

In [ ]:
attrs['ciscoRouter']['62.183.0.103']['inUtilization'].keys()

In [ ]:
for cls in res.model.unique():
    